In [103]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load the training data into feature matrix, class labels, and event ids:

In [104]:
from proj1_helpers import *
DATA_TRAIN_PATH = '../data/train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)
tX.shape

(250000, 30)

## Do your thing crazy machine learning thing here :) ...

In [105]:
from my_helper import *
from implementations import *

In [106]:
# pre-process
# Maybe subtract min? how do we treat values like -999?
std = np.std(tX, axis=0)
tX = tX / std
std.shape

(30,)

In [107]:
# required implementations

max_iters = 50
# least square
# gamma = 0.0000001
# logistic
gamma = 0.01 
batch_size = 32
lambda_ = 0.1
# initial_w = np.random.rand(tX.shape[1]) * 2 - 1
# weights, loss = least_squares_GD(y, tX, initial_w, max_iters, gamma)
# weights, loss = least_squares_SGD(y, tX, initial_w, batch_size, max_iters, gamma)
# weights, loss = least_squares(y, tX)
# weights, loss = ridge_regression(y, tX, lambda_)
# weights, loss = logistic_regression_GD(y, tX, initial_w, max_iters, gamma)
# weights, loss = reg_logistic_regression_SGD(y, tX, lambda_, initial_w, batch_size, max_iters, gamma)

In [108]:
# polynomial fit

ratio = 0.9
degree = 7
seed = 6
k_fold = 4

x_train, y_train, x_val, y_val = split_data(tX, y, ratio, seed)
x_train_poly = build_poly(x_train, degree)
x_val_poly = build_poly(x_val, degree)
initial_w = np.zeros(tX.shape[1]*degree)

lambdas = np.logspace(-5, 0, 30)

for lambda_ in lambdas:
    weights, loss = ridge_regression(y_train, x_train_poly, lambda_)

    y_train_pred = predict_labels(weights, x_train_poly)
    train_acc = cal_acc(y_train, y_train_pred)
    y_val_pred = predict_labels(weights, x_val_poly)
    val_acc = cal_acc(y_val, y_val_pred)

    print("Lambda {labd}, Train accuracy {ta}, Validation Accuracy {va}".format(labd=lambda_, ta=train_acc, va=val_acc))

Ridge Regression: loss=0.2985015609808699
Lambda 1e-05, Train accuracy 0.7921377777777778, Validation Accuracy 0.79316
Ridge Regression: loss=0.29893844163494243
Lambda 1.4873521072935119e-05, Train accuracy 0.7916355555555555, Validation Accuracy 0.79312
Ridge Regression: loss=0.299500537553365
Lambda 2.21221629107045e-05, Train accuracy 0.7912, Validation Accuracy 0.79204
Ridge Regression: loss=0.3001631836394837
Lambda 3.290344562312671e-05, Train accuracy 0.7906622222222223, Validation Accuracy 0.79104
Ridge Regression: loss=0.3008782202251289
Lambda 4.893900918477499e-05, Train accuracy 0.7899955555555556, Validation Accuracy 0.7904
Ridge Regression: loss=0.30159740112681827
Lambda 7.278953843983146e-05, Train accuracy 0.7890933333333333, Validation Accuracy 0.78908
Ridge Regression: loss=0.3022959601964204
Lambda 0.00010826367338740541, Train accuracy 0.7882533333333334, Validation Accuracy 0.78832
Ridge Regression: loss=0.3029820568718427
Lambda 0.0001610262027560939, Train accu

In [109]:
# best para

ratio = 0.9
degree = 8
lambda_ = 5.2983169062837125e-05
seed = 6

x_poly = build_poly(tX, degree)
weights, loss = ridge_regression(y, x_poly, lambda_)

pred = predict_labels(weights, x_poly)
acc = cal_acc(y, pred)

print(acc)

Ridge Regression: loss=0.29976130226731296
0.791032


## Generate predictions and save ouput in csv format for submission:

In [110]:
DATA_TEST_PATH = '../data/test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)
# pre-process
tX_test = tX_test / std

In [111]:
OUTPUT_PATH = 'predictions.csv' # TODO: fill in desired name of output file for submission
# y_pred = predict_labels(weights, tX_test)
# for polynomial fit
x_test_poly = build_poly(tX_test, degree)
y_pred = predict_labels(weights, x_test_poly)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)